In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pmd_beamphysics.wakefields import FlatResistiveWallImpedance

## Basic Usage

Create an impedance object by specifying the geometry and material properties:

In [ ]:
imp = FlatResistiveWallImpedance(
    half_gap=4.5e-3,  # m (half-gap between plates)
    conductivity=2.4e7,  # S/m (DC conductivity)
    relaxation_time=8e-15,  # s (Drude relaxation time)
)

imp

The characteristic length scale $s_0$ is computed automatically:

$$s_0 = \left( \frac{2a^2}{Z_0 \sigma_0} \right)^{1/3}$$

In [ ]:
print(f"Characteristic length s₀ = {imp.s0*1e6:.2f} µm")

## Computing Impedance $Z(k)$

The longitudinal impedance is computed by integrating the surface impedance kernel over all transverse modes.

In [ ]:
ks = np.linspace(0, 3e5, 100)
Zk = imp.impedance(ks)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(ks * 1e-3, np.real(Zk), label=r"Re[$Z(k)$]")
ax.plot(ks * 1e-3, np.imag(Zk), label=r"Im[$Z(k)$]")
ax.set_xlabel(r"$k$ (1/mm)")
ax.set_ylabel(r"$Z(k)$ (Ω/m)")
ax.legend()
ax.set_title("Longitudinal Impedance")
plt.tight_layout()

The built-in plot method provides a quick visualization:

In [ ]:
imp.plot_impedance(k_max=5e5, n_points=200)

## Computing Wakefield $W(z)$

The wakefield is computed from the real part of the impedance using a cosine transform:

$$W(z) = \frac{2}{\pi} \int_0^{k_{\max}} \text{Re}[Z(k)] \cos(kz) \, dk$$

In [ ]:
zs = np.linspace(0, 200e-6, 30)
Wz = imp.wakefield(zs, k_max=1e6)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(zs * 1e6, Wz * 1e-12)
ax.set_xlabel(r"$z$ (µm)")
ax.set_ylabel(r"$W(z)$ (V/pC/m)")
ax.set_title("Longitudinal Wakefield")
plt.tight_layout()

The built-in plot method:

In [ ]:
imp.plot_wakefield(z_max=200e-6, n_points=40, k_max=1e6)

## Low-Level Functions

The module also provides low-level functions for more control:

In [ ]:
from pmd_beamphysics.wakefields.resistive_wall_impedance import (
    ac_conductivity,
    longitudinal_impedance,
    wakefield_from_impedance,
)

### AC Conductivity

The Drude model for frequency-dependent conductivity:

$$\sigma(k) = \frac{\sigma_0}{1 - i k c \tau}$$

In [ ]:
sigma0 = 2.4e7  # S/m
ctau = 2.4e-6  # m

ks = np.linspace(0, 1e6, 200)
sigma_ac = ac_conductivity(ks, sigma0, ctau)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(ks * 1e-3, np.real(sigma_ac) / sigma0, label=r"Re[$\sigma$]/$\sigma_0$")
ax.plot(ks * 1e-3, np.imag(sigma_ac) / sigma0, label=r"Im[$\sigma$]/$\sigma_0$")
ax.set_xlabel(r"$k$ (1/mm)")
ax.set_ylabel(r"Normalized conductivity")
ax.legend()
ax.set_title("AC Conductivity (Drude Model)")
plt.tight_layout()

### Direct Impedance Calculation

Use `longitudinal_impedance` directly with explicit parameters:

In [ ]:
a = 4.5e-3  # half-gap [m]
sigma0 = 2.4e7  # conductivity [S/m]
ctau = 2.4e-6  # relaxation distance [m]

k_test = 1e5  # 1/m
Zk_test = longitudinal_impedance(k_test, a, sigma0, ctau)
print(f"Z({k_test:.0e} 1/m) = {Zk_test:.4f} Ω/m")

### Wakefield from Custom Impedance Function

The `wakefield_from_impedance` function accepts any callable that returns $Z(k)$:

In [ ]:
from functools import partial

# Create a custom impedance function
my_Zk = partial(longitudinal_impedance, a=4.5e-3, sigma0=2.4e7, ctau=2.4e-6)

# Compute wakefield at a single point
z_test = 50e-6  # m
Wz_test = wakefield_from_impedance(z_test, my_Zk, k_max=1e6)
print(f"W({z_test*1e6:.0f} µm) = {Wz_test:.2e} V/C/m = {Wz_test*1e-12:.2f} V/pC/m")

## Performance Notes

The impedance calculation involves nested numerical integration (over k and a transverse integration variable), which can be slow for large arrays. For production use with many particles, consider:

1. Pre-computing Z(k) on a grid and interpolating
2. Using the `ResistiveWallWakefield` class with pseudomode fits for faster evaluation

See the [resistive_wall_wakefield](resistive_wall_wakefield.ipynb) notebook for the pseudomode approach.

In [ ]:
%%timeit -n 1 -r 3
# Time a typical impedance calculation
ks = np.linspace(0, 3e5, 50)
Zk = imp.impedance(ks)